In [343]:
from numpy.random import RandomState
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import jieba
import jieba.analyse

df = pd.read_csv('movies.csv')
#str -> dict
df['類別'] = df['類別'].apply(eval)

# list first catagory
def list_genres(x):
    for d in x.values():
        return(d)

df['類別'] = df['類別'].apply(list_genres)

In [344]:
#split overview
overview = []
for d in df['劇情介紹']:
    keywords = jieba.analyse.extract_tags(d, topK=8)
    overview.append(keywords)

df['關鍵字'] = overview
df['關鍵字'].head()


0              [電影, 這部, 小小, 動畫, 超級, 格魯, 導演, 克里斯]
1             [蝙蝠, 一個, 布魯斯, 韋恩, 並從, 中檢視, 靈魂, 伯派]
2             [奧斯卡, 雷托領, 治療, 實驗, 結果, 瑞德, 比斯, 宇宙]
3    [捍衛戰士, 任務, 1986, 湯姆克魯斯, 獨行俠, 派拉蒙, 金斯基, 珍妮佛]
4              [魔鬼, 星未來, 童星, 保羅, 續集, 他們, 特曼, 系列]
Name: 關鍵字, dtype: object

In [346]:
import re
df['keywords'] = ''
def bag_words(x):
    return (''.join(re.sub('[^\u4e00-\u9fa5]+', '', x['中文名稱'])) + ' ' + ''.join(x['類別']) + ' ' +  ' '.join(x['關鍵字']))
df['keywords'] = df.apply(bag_words, axis = 1)

df['keywords'].head()
#print(df['keywords'])

0             小小兵格魯的崛起 動畫 電影 這部 小小 動畫 超級 格魯 導演 克里斯
1                 蝙蝠俠 動作 蝙蝠 一個 布魯斯 韋恩 並從 中檢視 靈魂 伯派
2                 魔比斯 動作 奧斯卡 雷托領 治療 實驗 結果 瑞德 比斯 宇宙
3    捍衛戰士獨行俠 動作 捍衛戰士 任務 1986 湯姆克魯斯 獨行俠 派拉蒙 金斯基 珍妮佛
4              魔鬼剋星未來世 動作 魔鬼 星未來 童星 保羅 續集 他們 特曼 系列
Name: keywords, dtype: object

In [347]:
#split Data to train and test
rng = RandomState()
train = df.sample(frac=0.9173, random_state=rng)
test = df.loc[~df.index.isin(train.index)]
test['類別'].to_csv('test.csv')

In [350]:
# 將文件中的詞語轉換為詞頻矩陣
cv = CountVectorizer()
cv_train = cv.fit_transform(train['keywords'])
cv_test = cv.transform(test['keywords'])
#cv_mx.toarray()

In [351]:
from sklearn.feature_extraction.text import TfidfTransformer
#類調用
transformer = TfidfTransformer()

#將詞頻矩陣X統計成TF-IDF值

tfidf_train = transformer.fit_transform(cv_train)
tfidf_test = transformer.transform(cv_test)
#print(tfidf)

In [366]:
from sklearn.neighbors import KNeighborsClassifier
knn  = KNeighborsClassifier(n_neighbors=11,p=2,n_jobs=2)
knn.fit(tfidf_train,train['類別'])

KNeighborsClassifier(n_jobs=2, n_neighbors=13)

In [367]:
knn.score(tfidf_train,train['類別'])

0.7967171717171717

In [360]:
knn.predict(tfidf_test)

array(['動作', '動作', '動作', '動畫', '劇情', '劇情', '動作', '動作', '動作', '動作', '動作',
       '動作', '動作', '動作', '動作', '動作', '劇情', '動作', '劇情', '動作', '動作', '動作',
       '劇情', '動作', '動作', '動作', '劇情', '動作', '動作', '動作', '劇情', '動作', '動作',
       '動作', '動作', '動作', '動作', '動作', '動作', '動作', '動作', '動作', '動作', '動作',
       '動作', '動作', '動作', '動作', '動作', '劇情', '動作', '奇幻', '動作', '動作', '動作',
       '動作', '愛情', '劇情', '動畫', '溫馨/家庭', '動作', '劇情', '動畫', '影展', '動作',
       '喜劇', '冒險', '動作', '冒險', '動作', '冒險', '冒險', '溫馨/家庭', '動作', '動作',
       '動作', '劇情', '冒險', '劇情', '動作', '劇情', '動作', '動作', '動作', '冒險', '動作',
       '動作', '劇情', '動作', '恐怖', '動作', '冒險', '溫馨/家庭', '動作', '動作', '動畫',
       '動作', '恐怖', '恐怖', '動作', '科幻', '劇情', '動作', '劇情', '動作', '科幻', '劇情',
       '科幻', '科幻', '劇情', '動作', '動作', '動作', '動作', '科幻', '動作', '恐怖', '奇幻',
       '劇情', '奇幻', '劇情', '動畫', '奇幻', '奇幻', '奇幻', '劇情', '劇情', '音樂/歌舞',
       '奇幻', '奇幻', '冒險', '奇幻', '音樂/歌舞', '劇情', '動作', '喜劇', '奇幻', '奇幻',
       '動作', '冒險', '劇情', '動作', '動作', '恐怖', '恐怖', '劇情', '劇情', '愛情',

In [361]:
knn.score(tfidf_test,test['類別'])

0.644